## Paper: Efficiently solving the practical vehicle routing problem

Source: `Duan et al., ‘Efficiently Solving the Practical Vehicle Routing Problem’.`

In [110]:
import pickle
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset
from pathlib import Path

In [138]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Device", device)

Device cpu


In [139]:
INSTANCE_PATH = Path("data")
RESULTS_PATH = Path("results")

In [140]:
with open(INSTANCE_PATH / 'dataset.pkl', 'rb') as f:
    graphs = pickle.load(f)

In [141]:
graphs[0]

(array([[0.5       , 0.5       , 0.        ],
        [0.06505159, 0.94888554, 2.        ],
        [0.96563203, 0.80839735, 4.        ],
        [0.30461377, 0.09767211, 8.        ],
        [0.68423303, 0.44015249, 7.        ],
        [0.12203823, 0.49517691, 9.        ],
        [0.03438852, 0.9093204 , 8.        ],
        [0.25877998, 0.66252228, 5.        ],
        [0.31171108, 0.52006802, 2.        ],
        [0.54671028, 0.18485446, 5.        ],
        [0.96958463, 0.77513282, 8.        ]]),
 array([-1,  0,  1,  2,  1,  0,  0,  0,  2,  2,  1]))

In [142]:
def get_distance(pointA, pointB):
    # Efficient way to calculate the euclidean distance
    return np.linalg.norm(pointA - pointB)

In [164]:
def pre_process_graph(graph):
    nodes, label = graph
    
    label = torch.tensor(label + 1, dtype=torch.int64)
    graph = torch.tensor(nodes[:, :-1], dtype=torch.float32)
    demand = torch.tensor(nodes[:, -1], dtype=torch.float32)
    distance = np.zeros((nodes.shape[0], nodes.shape[0]))

    for i in range(len(distance)):
        for j in range(i + 1, len(distance)):
            d = get_distance(graph[i], graph[j])
            distance[i][j] = d
            distance[j][i] = d
            
    distance = torch.tensor(distance, dtype=torch.float32)
            
    return graph, demand, distance, label

In [165]:
class GraphDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.graphs = []
        self.demands = []
        self.distances = []
        self.labels = []
        
        for i in range(len(data)):
            processed = pre_process_graph(data[i])
            
            self.graphs.append(processed[0])
            self.demands.append(processed[1])
            self.distances.append(processed[2])
            self.labels.append(processed[3])

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        graph = self.graphs[idx]
        demand = self.demands[idx]
        distance = self.distances[idx]
        label = self.labels[idx]
        
        return graph, demand, distance, label
    
    def __repr__(self):
        return f"GraphDataset(graphs={len(self.graphs)})"

The dataset should still be split in a train and test dataset

In [166]:
graph_dataset = GraphDataset(graphs)
graph_dataset

GraphDataset(graphs=2000)

In [167]:
train_dataloader = DataLoader(graph_dataset, batch_size=64, shuffle=True)

## Attention Encoder

In [168]:
class AttentionEncoder(nn.Module):
    def __init__(self, hidden_dim):
        super(AttentionEncoder, self).__init__()
        self.hidden_dim = hidden_dim

    def forward(self, x, neighbor):
        '''
        @param x: (batch_size, node_num, hidden_dim)
        @param neighbor: (batch_size, node_num, k, hidden_dim)
        '''
        # scaled dot-product attention
        x = x.unsqueeze(2)
        neighbor = neighbor.permute(0, 1, 3, 2)
        attn_score = F.softmax(torch.matmul(x, neighbor) / np.sqrt(self.hidden_dim), dim=-1) # (batch_size, node_num, 1, k)
        weighted_neighbor = attn_score * neighbor
        
        # aggregation
        agg = x.squeeze(2) + torch.sum(weighted_neighbor, dim=-1)
        
        return agg

class AttentionPointer(nn.Module):
    def __init__(self, hidden_dim, use_tanh=False, use_cuda=False):
        super(AttentionPointer, self).__init__()
        self.hidden_dim = hidden_dim
        self.use_tanh = use_tanh

        self.project_hidden = nn.Linear(hidden_dim, hidden_dim)
        self.project_x = nn.Conv1d(hidden_dim, hidden_dim, 1, 1)
        self.C = 10
        self.tanh = nn.Tanh()

        v = torch.FloatTensor(hidden_dim)
        if use_cuda:
            v = v.cuda()
        self.v = nn.Parameter(v)
        self.v.data.uniform_(-(1. / math.sqrt(hidden_dim)) , 1. / math.sqrt(hidden_dim))

    def forward(self, hidden, x):
        '''
        @param hidden: (batch_size, hidden_dim)
        @param x: (node_num, batch_size, hidden_dim)
        '''
        x = x.permute(1, 2, 0)
        q = self.project_hidden(hidden).unsqueeze(2)  # batch_size x hidden_dim x 1
        e = self.project_x(x)  # batch_size x hidden_dim x node_num 
        # expand the hidden by node_num
        # batch_size x hidden_dim x node_num
        expanded_q = q.repeat(1, 1, e.size(2)) 
        # batch x 1 x hidden_dim
        v_view = self.v.unsqueeze(0).expand(expanded_q.size(0), len(self.v)).unsqueeze(1)
        # (batch_size x 1 x hidden_dim) * (batch_size x hidden_dim x node_num)
        u = torch.bmm(v_view, self.tanh(expanded_q + e)).squeeze(1)
        if self.use_tanh:
            logits = self.C * self.tanh(u)
        else:
            logits = u  
        return e, logits

## GCN

In [174]:
class GCN(nn.Module):
    def __init__(self,
                 node_hidden_dim,
                 edge_hidden_dim,
                 gcn_num_layers,
                 k):
        super(GCN, self).__init__()

        self.node_hidden_dim = node_hidden_dim
        self.edge_hidden_dim = edge_hidden_dim
        self.gcn_num_layers = gcn_num_layers
        self.k = k
        
        self.W1 = nn.Linear(2, self.node_hidden_dim)      # node_W1
        self.W2 = nn.Linear(2, self.node_hidden_dim // 2) # node_W2
        self.W3 = nn.Linear(1, self.node_hidden_dim // 2) # node_W3
        self.W4 = nn.Linear(1, self.edge_hidden_dim // 2) # edge_W4
        self.W5 = nn.Linear(1, self.edge_hidden_dim // 2) # edge_W5
        
        self.node_embedding = nn.Linear(self.node_hidden_dim, self.node_hidden_dim, bias=False) # Eq5
        self.edge_embedding = nn.Linear(self.edge_hidden_dim, self.edge_hidden_dim, bias=False) # Eq6

        self.gcn_layers = nn.ModuleList([GCNLayer(self.node_hidden_dim) for i in range(self.gcn_num_layers)])
        
        self.relu = nn.ReLU()

    def adjacency(self, m):
        '''
        @param m: distance (node_num, node_num)
        '''
        a = torch.zeros_like(m)
        idx = torch.argsort(m, dim=1)[:, 1:(self.k+1)]
        a.scatter_(1, idx, 1)
        a.fill_diagonal_(-1)

        return a

    def find_neighbors(self, m):
        ''' find index of neighbors for each node
        @param m: distance (batch_size, node_num, node_num)
        '''
        neighbor_idx = []
        for i in range(m.shape[0]):
            idx = torch.argsort(m[i, :, :], dim=1)[:, 1:(self.k+1)].numpy()
            neighbor_idx.append(idx)
        return torch.LongTensor(neighbor_idx).to(device)

    def forward(self, x_c, x_d, m):
        '''
        @param x_c: coordination (batch_size, node_num(N+1), 2)
        @param x_d: demand (batch_size, node_num(N+1))
        @param m: distance (batch_size, node_num(N+1), node_num(N+1))
        '''
        # Eq 2
        x0 = self.relu(self.W1(x_c[:, :1, :])) # (batch_size, 1, node_hidden_dim)
        xi = self.relu(torch.cat((self.W2(x_c[:, 1:, :]), self.W3(x_d.unsqueeze(2)[:, 1:, :])), dim=-1)) # (batch_size, node_num(N), node_hidden_dim)
        x = torch.cat((x0, xi), dim=1)
        # Eq 3
        a = torch.Tensor([self.adjacency(m[i, :, :]).numpy() for i in range(m.shape[0])]).to(device)
        # Eq 4
        y = self.relu(torch.cat((self.W4(m.unsqueeze(3)), self.W5(a.unsqueeze(3))), dim=-1))
        # Eq 5
        h_node = self.node_embedding(x)
        # Eq 6
        h_edge = self.edge_embedding(y)

        # index of neighbors
        N = self.find_neighbors(m)

        # GCN layers
        for gcn_layer in self.gcn_layers:
            h_node, h_edge = gcn_layer(h_node, h_edge, N)

        return h_node, h_edge


class GCNLayer(nn.Module):
    def __init__(self, hidden_dim):
        super(GCNLayer, self).__init__()

        # node GCN layers
        self.W_node = nn.Linear(hidden_dim, hidden_dim)
        self.V_node_in = nn.Linear(hidden_dim, hidden_dim)
        self.V_node = nn.Linear(2 * hidden_dim, hidden_dim)
        self.attn = AttentionEncoder(hidden_dim)
        self.relu = nn.ReLU()
        self.ln1_node = nn.LayerNorm(hidden_dim)
        self.ln2_node = nn.LayerNorm(hidden_dim)

        # edge GCN layers
        self.W_edge = nn.Linear(hidden_dim, hidden_dim)
        self.V_edge_in = nn.Linear(hidden_dim, hidden_dim)
        self.V_edge = nn.Linear(2 * hidden_dim, hidden_dim)
        self.W1_edge = nn.Linear(hidden_dim, hidden_dim)
        self.W2_edge = nn.Linear(hidden_dim, hidden_dim)
        self.W3_edge = nn.Linear(hidden_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.ln1_edge = nn.LayerNorm(hidden_dim)
        self.ln2_edge = nn.LayerNorm(hidden_dim)

        self.hidden_dim = hidden_dim

    def forward(self, x, e, neighbor_index):
        '''
        @param x: (batch_size, node_num(N+1), node_hidden_dim)
        @param e: (batch_size, node_num(N+1), node_num(N+1), edge_hidden_dim)
        @param neighbor_index: (batch_size, node_num(N+1), k)
        '''
        # node embedding
        batch_size, node_num = x.size(0), x.size(1)
        node_hidden_dim = x.size(-1)
        t = x.unsqueeze(1).repeat(1, node_num, 1, 1)

        neighbor_index = neighbor_index.unsqueeze(3).repeat(1, 1, 1, node_hidden_dim)
        neighbor = t.gather(2, neighbor_index)
        neighbor = neighbor.view(batch_size, node_num, -1, node_hidden_dim)
        
        # Eq 7/9
        h_nb_node = self.ln1_node(x + self.relu(self.W_node(self.attn(x, neighbor))))
        # Eq 12, Eq 8
        h_node = self.ln2_node(h_nb_node + self.relu(self.V_node(torch.cat([self.V_node_in(x), h_nb_node], dim=-1))))

        # edge embedding
        x_from = x.unsqueeze(2).repeat(1, 1, node_num, 1)
        x_to = x.unsqueeze(1).repeat(1, node_num, 1, 1)
        # Eq 7/10, Eq 11
        h_nb_edge = self.ln1_edge(e + self.relu(self.W_edge(self.W1_edge(e) + self.W2_edge(x_from) + self.W3_edge(x_to))))
        # Eq 13, Eq 8
        h_edge = self.ln2_edge(h_nb_edge + self.relu(self.V_edge(torch.cat((self.V_edge_in(e), h_nb_edge), dim=-1))))

        return h_node, h_edge

In [175]:
NODE_HIDDEN = 2
EDGE_HIDDEN = 2
GCN_LAYER = 2
k = 1

learning_rate = 1e-4
weight_decay = 0.96

model = GCN(node_hidden_dim=NODE_HIDDEN,
            edge_hidden_dim=EDGE_HIDDEN,
            gcn_num_layers=GCN_LAYER,
            k=k).to(device)

optimizer = torch.optim.Adam(model.parameters(),
                             lr=learning_rate,
                             weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

In [176]:
x_c, x_d, m, y = next(iter(train_dataloader))

In [177]:
x_c.shape, x_d.shape, m.shape

(torch.Size([64, 11, 2]), torch.Size([64, 11]), torch.Size([64, 11, 11]))

In [184]:
model.eval()

with torch.no_grad():
    x_c, x_d, m = x_c.to(device), x_d.to(device), m.to(device)
    h_node, h_edge = model(x_c, x_d, m)

In [185]:
h_node.shape, h_edge.shape

(torch.Size([64, 11, 2]), torch.Size([64, 11, 11, 2]))